# Demand Profiling

This notebook takes a set of input demands, profiles and optional sector system to produce a set of profiled demands.
See the example inputs for file format.

In [1]:
import pandas as pd
import plotly.express as px

# Inputs

In [29]:
# define input folder
input_path = 'input/'

# define output folder
output_path = 'output/'

# dicionary to store demand, profile pairs
demand_inputs = {'car_demand.csv': 'car_profile.csv',
                  'truck_demand.csv': 'truck_profile.csv'  
                 }

sector_file = 'sector.csv'
# sector_file = None


# Create Plot of Profiles for Inspection (Optional)

In [40]:
# create a plot of profiles
selected_file = 'car_profile.csv'

profile = pd.read_csv(f'{input_path}{selected_file}')
# Melting the dataframe to convert it from wide to long format
profile = profile.melt(id_vars=['origin', 'destination'], var_name='time', value_name='demand proportion')
# Creating a new column for the profile name combining 'origin' and 'destination'
profile['profile'] = profile['origin'].astype(str) + '-' + profile['destination'].astype(str)
# Plotting with Plotly Express
fig = px.line(profile, x='time', y='demand proportion', color='profile', title='Profiles by Origin and Destination')

# Show the figure
fig.show()

# Create Profiled Demand

In [36]:
for demand_file, profile_file in demand_inputs.items():
    # read demand and profile
    demand = pd.read_csv(f'{input_path}{demand_file}')
    profile = pd.read_csv(f'{input_path}{profile_file}')
    
    if sector_file:
        # read sector
        sector = pd.read_csv(f'{input_path}{sector_file}')
        
        # merge sector origin and destination
        demand = demand.merge(sector, left_on='origin', right_on='zone')
        demand = demand.merge(sector, left_on='destination', right_on='zone', suffixes=('_origin', '_destination'))

        # drop the zone_origin and zone_destination columns
        demand = demand.drop(columns=['zone_origin', 'zone_destination'])

        # merge demand and profile
        demand = demand.merge(profile, left_on=['sector_origin', 'sector_destination'], right_on=['origin', 'destination'], suffixes=('', '_profile'))
        # drop the origin and destination columns
        demand = demand.drop(columns=['origin_profile', 'destination_profile'])

        # multiply demand by each time column
        for time in profile.columns[2:]:
            demand[f'trips_{time}'] = demand['trips'] * demand[time]

    # else, merge demand and profile
    else:
        demand = demand.merge(profile, left_on=['origin', 'destination'], right_on=['origin', 'destination'], suffixes=('', '_profile'))
        # drop the origin and destination columns
        demand = demand.drop(columns=['origin_profile', 'destination_profile'])

        # multiply demand by each time column
        for time in profile.columns[2:]:
            demand[f'trips_{time}'] = demand['trips'] * demand[time]


    # sum the trips and trips by time to check if the sum is the same as the original demand (should be zero or very close to zero)
    demand['trips_profiles'] = demand[[f'trips_{time}' for time in profile.columns[2:]]].sum(axis=1)
    print((demand['trips'] - demand['trips_profiles']).sum())

    # remove all columns other than origin, destinatio and trips by time
    demand = demand[['origin', 'destination'] + [f'trips_{time}' for time in profile.columns[2:]]]

    # strip out trips from the time column names
    demand.columns = demand.columns.str.replace('trips_', '')

    # save the demand
    demand.to_csv(f'{output_path}{demand_file.replace(".csv", "_profiled.csv")}', index=False)
    print(f'{output_path}{demand_file.replace(".csv", "_profiled.csv")} file saved!')


-3.2900002722158206e-07
output/car_demand_profiled.csv file saved!
0.0
output/truck_demand_profiled.csv file saved!
